In [1]:
# to ensure that the logging statements are shown in juypter output, run this cell
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)

In [24]:
from secfsdstools.update import update

update()

2024-08-07 06:34:30,159 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2024-08-07 06:34:30,173 [INFO] updateprocess  Check if new report zip files are available...
2024-08-07 06:34:30,228 [INFO] updateprocess  check if there are new files to download from sec.gov ...
2024-08-07 06:34:30,925 [INFO] updateprocess  start to transform to parquet format ...
2024-08-07 06:34:30,938 [INFO] updateprocess  start to index parquet files ...


No rapid-api-key is set: 
If you are interested in daily updates, please have a look at https://rapidapi.com/hansjoerg.wingeier/api/daily-sec-financial-statement-dataset


In [3]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.e_filter.rawfiltering import ReportPeriodAndPreviousPeriodRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter

# the unique identifier for apple's 10-Q Q2 report of 2024
apple_10q_q2_2024_adsh = "0000320193-24-000069"

# us a Collector to grab the data of the 10-K report. filter for balancesheet information
collector: SingleReportCollector = SingleReportCollector.get_report_by_adsh(
      adsh=apple_10q_q2_2024_adsh
)  
rawdatabag = collector.collect() # load the data from the disk

2025-02-10 16:30:03,763 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-10 16:30:04,435 [INFO] updateprocess  Launching data update process ...
2025-02-10 16:30:04,463 [INFO] task_framework  Starting process SecDownloadingProcess
2025-02-10 16:30:04,465 [INFO] secdownloading_process  reading table in main page: https://www.sec.gov/dera/data/financial-statement-data-sets.html
2025-02-10 16:30:06,060 [INFO] task_framework  Starting process ToParquetTransformerProcess
2025-02-10 16:30:06,065 [INFO] task_framework  Starting process ReportParquetIndexerProcess
2025-02-10 16:30:06,104 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


In [4]:
sub_df = rawdatabag.sub_df
num_df = rawdatabag.num_df
pre_df = rawdatabag.pre_df

In [5]:
sub_df = sub_df[sub_df.adsh=="0000320193-24-000069"]
num_df = num_df[num_df.adsh=="0000320193-24-000069"]
pre_df = pre_df[pre_df.adsh=="0000320193-24-000069"]

In [6]:
print(sub_df.shape)
print(num_df.shape)
print(pre_df.shape)

(1, 36)
(433, 10)
(95, 10)


In [7]:
num_df.coreg = None

In [8]:
num_df = num_df[num_df.coreg.isna()]

In [9]:
num_df.uom.value_counts()

USD       419
shares     14
Name: uom, dtype: int64

In [14]:
# we want to keep all values with uoms that are  not in upper case
mask_has_lower = ~num_df.uom.str.isupper()

# currency has always 3 letters, so we want to keep everything that has a different length
mask_is_none_currency = num_df.uom.str.len() != 3

# keep USD
mask_usd_only = num_df.uom == "USD"

num_df = num_df[mask_has_lower | mask_is_none_currency | mask_usd_only]

In [15]:
print(num_df.shape)

(344, 10)


In [16]:
# get the value of the "period" column for the entry in sub_df
# (there is only one entry left, since filtered for a certain adsh)
period = sub_df.iloc[0].period

# mask the datapoints for the current period
mask_current = num_df.ddate == period

# since period and ddate are actually numbers in the form of YYYYMMDD
# we can simply subtract 10'000 to get the previous year
# !!!! ATTENTION !!! 
# When the period is end of February, we have to consider the leap years!
mask_previous = num_df.ddate == (period - 10000)

num_df = num_df[mask_current | mask_previous]

In [17]:
print(num_df.shape)

(344, 10)


In [18]:
num_df = num_df[num_df.qtrs.isin([1,2])]

In [19]:
num_df[num_df.tag=="RevenueFromContractWithCustomerExcludingAssessedTax"]

,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote
2,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,1,USD,BusinessSegments=GreaterChinaSegment;,None,1.637200e+10,None
15,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,2,USD,,None,2.103280e+11,None
39,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,2,USD,BusinessSegments=RestOfAsiaPacificSegment;,None,1.688500e+10,None
65,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20230331,1,USD,,None,9.483600e+10,None
70,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20230331,1,USD,ProductOrService=IPad;,None,6.670000e+09,None
77,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,1,USD,ProductOrService=IPad;,None,5.559000e+09,None
81,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20230331,2,USD,ProductOrService=Mac;,None,1.490300e+10,None
98,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,1,USD,,None,9.075300e+10,None
100,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20230331,2,USD,ProductOrService=Service;,None,4.167300e+10,None
107,0000320193-24-000069,RevenueFromContractWithCustomerExcludingAssessedTax,us-gaap/2023,20240331,2,USD,BusinessSegments=GreaterChinaSegment;,None,3.719100e+10,None


In [20]:
print(num_df.shape)

(263, 10)


In [21]:
pre_df = pre_df[pre_df.stmt == 'IS']

In [22]:
print(pre_df.shape)

(15, 10)


In [23]:
pre_num_df = pd.merge(num_df,
                      pre_df,
                      on=['adsh', 'tag', 'version'])

In [24]:
print(pre_num_df.shape)

(144, 17)


In [25]:
# mask the entries with the negating flag set and inverse the value column
pre_num_df.loc[pre_num_df.negating == 1, 'value'] = -pre_num_df.value

In [26]:
pre_num_df = pre_num_df[['tag', 'line', 'report', 'segments', 'uom', 'value', 'ddate', 'qtrs']]

In [27]:
pivot_df = pre_num_df.pivot_table(
                index=['tag','report', 'line', 'segments', 'uom'],
                columns=['qtrs', 'ddate'],
                values='value')

In [28]:
sort_df = pivot_df.sort_values(['report', 'line'])

In [37]:
sort_df

qtrs                                                                                                                                                                     1                           2              
ddate                                                                                                                                                             20230331      20240331      20230331      20240331
tag                                                                                         report line segments                                      uom                                                           
RevenueFromContractWithCustomerExcludingAssessedTax                                         2      7                                                  USD     9.483600e+10  9.075300e+10  2.119900e+11  2.103280e+11
                                                                                                        BusinessSegments=AmericasSegment;             USD     3.778400e+10  3.727300e+10  8.706200e+10  8.770300e+10
                                                                                                        BusinessSegments=EuropeSegment;               USD     2.394500e+10  2.412300e+10  5.162600e+10  5.452000e+10
                                                                                                        BusinessSegments=GreaterChinaSegment;         USD     1.781200e+10  1.637200e+10  4.171700e+10  3.719100e+10
                                                                                                        BusinessSegments=JapanSegment;                USD     7.176000e+09  6.262000e+09  1.393100e+10  1.402900e+10
                                                                                                        BusinessSegments=RestOfAsiaPacificSegment;    USD     8.119000e+09  6.723000e+09  1.765400e+10  1.688500e+10
                                                                                                        ProductOrService=IPad;                        USD     6.670000e+09  5.559000e+09  1.606600e+10  1.258200e+10
                                                                                                        ProductOrService=IPhone;                      USD     5.133400e+10  4.596300e+10  1.171090e+11  1.156650e+11
                                                                                                        ProductOrService=Mac;                         USD     7.168000e+09  7.451000e+09  1.490300e+10  1.523100e+10
                                                                                                        ProductOrService=Product;                     USD     7.392900e+10  6.688600e+10  1.703170e+11  1.633440e+11
                                                                                                        ProductOrService=Service;                     USD     2.090700e+10  2.386700e+10  4.167300e+10  4.698400e+10
                                                                                                        ProductOrService=WearablesHomeandAccessories; USD     8.757000e+09  7.913000e+09  2.223900e+10  1.986600e+10
CostOfGoodsAndServicesSold                                                                  2      8                                                  USD     5.286000e+10  4.848200e+10  1.196820e+11  1.132020e+11
                                                                                                        ProductOrService=Product;                     USD     4.679500e+10  4.242400e+10  1.075600e+11  1.008640e+11
                                                                                                        ProductOrService=Service;                     USD     6.065000e+09  6.058000e+09  1.212200e+10  1.233800e+10
GrossProfit                                                                                 2      9                                                  USD     4.197600

In [47]:
sort_df[sort_df.index.get_level_values('segments').isin(['', 'ProductOrService=Service;', 'ProductOrService=Product;'])]

qtrs                                                                                                                                                 1                           2              
ddate                                                                                                                                         20230331      20240331      20230331      20240331
tag                                                                                         report line segments                  uom                                                           
RevenueFromContractWithCustomerExcludingAssessedTax                                         2      7                              USD     9.483600e+10  9.075300e+10  2.119900e+11  2.103280e+11
                                                                                                        ProductOrService=Product; USD     7.392900e+10  6.688600e+10  1.703170e+11  1.633440e+11
                                                                                                        ProductOrService=Service; USD     2.090700e+10  2.386700e+10  4.167300e+10  4.698400e+10
CostOfGoodsAndServicesSold                                                                  2      8                              USD     5.286000e+10  4.848200e+10  1.196820e+11  1.132020e+11
                                                                                                        ProductOrService=Product; USD     4.679500e+10  4.242400e+10  1.075600e+11  1.008640e+11
                                                                                                        ProductOrService=Service; USD     6.065000e+09  6.058000e+09  1.212200e+10  1.233800e+10
GrossProfit                                                                                 2      9                              USD     4.197600e+10  4.227100e+10  9.230800e+10  9.712600e+10
ResearchAndDevelopmentExpense                                                               2      11                             USD     7.457000e+09  7.903000e+09  1.516600e+10  1.559900e+10
SellingGeneralAndAdministrativeExpense                                                      2      12                             USD     6.201000e+09  6.468000e+09  1.280800e+10  1.325400e+10
OperatingExpenses                                                                           2      13                             USD     1.365800e+10  1.437100e+10  2.797400e+10  2.885300e+10
OperatingIncomeLoss                                                                         2      14                             USD     2.831800e+10  2.790000e+10  6.433400e+10  6.827300e+10
NonoperatingIncomeExpense                                                                   2      15                             USD     6.400000e+07  1.580000e+08 -3.290000e+08  1.080000e+08
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest 2      16                             USD     2.838200e+10  2.805800e+10  6.400500e+10  6.838100e+10
IncomeTaxExpenseBenefit                                                                     2      17                             USD     4.222000e+09  4.422000e+09  9.847000e+09  1.082900e+10
NetIncomeLoss                                                                               2      18                             USD     2.416000e+10  2.363600e+10  5.415800e+10  5.755200e+10
EarningsPerShareBasic                                                                       2      20                             USD     1.530000e+00  1.530000e+00  3.420000e+00  3.720000e+00
EarningsPerShareDiluted                                                                     2      21                             USD     1.520000e+00  1.530000e+00  3.410000e+00  3.710000e+00
WeightedAverageNumberOfSharesOutstandingBasic                                               2      23                             shares  1.